# 7. Cách Run với Docker

Docker giúp bạn đóng gói ứng dụng của mình cùng với tất cả các phụ thuộc cần thiết vào một container, đảm bảo rằng ứng dụng sẽ chạy nhất quán trên mọi môi trường. Dưới đây là hướng dẫn chi tiết để chạy project **Video Scoring UI React** bằng Docker.

## 7.1. Tạo Dockerfile

Đã có một file `Dockerfile` trong cấu trúc project của bạn. Dưới đây là nội dung mẫu cho `Dockerfile` để xây dựng và chạy ứng dụng React:

```dockerfile
# Sử dụng Node.js image chính thức từ Docker Hub
FROM node:18-alpine AS build

# Thiết lập thư mục làm việc trong container
WORKDIR /app

# Sao chép file package.json và package-lock.json
COPY package*.json ./

# Cài đặt các dependencies
RUN npm install

# Sao chép toàn bộ mã nguồn vào container
COPY . .

# Build ứng dụng cho production
RUN npm run build

# Sử dụng Nginx để phục vụ ứng dụng
FROM nginx:stable-alpine

# Sao chép file build từ stage trước vào thư mục mặc định của Nginx
COPY --from=build /app/build /usr/share/nginx/html

# Sao chép cấu hình Nginx nếu cần (tùy chọn)
# COPY nginx.conf /etc/nginx/nginx.conf

# Expose cổng mà Nginx sẽ lắng nghe
EXPOSE 80

# Khởi động Nginx
CMD ["nginx", "-g", "daemon off;"]
```

### Giải thích các bước trong Dockerfile:

1. **Stage Build:**
   - **FROM node:18-alpine AS build**: Sử dụng image Node.js phiên bản 18 trên nền Alpine Linux, nhẹ và phù hợp cho việc build ứng dụng.
   - **WORKDIR /app**: Thiết lập thư mục làm việc trong container.
   - **COPY package*.json ./**: Sao chép `package.json` và `package-lock.json` vào thư mục làm việc.
   - **RUN npm install**: Cài đặt các dependencies.
   - **COPY . .**: Sao chép toàn bộ mã nguồn vào container.
   - **RUN npm run build**: Build ứng dụng React cho production.

2. **Stage Production:**
   - **FROM nginx:stable-alpine**: Sử dụng Nginx để phục vụ ứng dụng.
   - **COPY --from=build /app/build /usr/share/nginx/html**: Sao chép kết quả build từ stage trước vào thư mục mặc định của Nginx.
   - **EXPOSE 80**: Mở cổng 80 để Nginx lắng nghe.
   - **CMD ["nginx", "-g", "daemon off;"]**: Khởi động Nginx khi container chạy.

## 7.2. Tạo `.dockerignore`

Để tối ưu hóa quá trình build Docker và tránh sao chép những file không cần thiết, hãy tạo một file `.dockerignore` trong thư mục gốc của project với nội dung sau:

```plaintext
node_modules
build
.dockerignore
Dockerfile
.git
.gitignore
```

## 7.3. Build Docker Image

Mở terminal và chuyển đến thư mục gốc của project (`ui_react`), sau đó chạy lệnh sau để build Docker image:

```bash
docker build -t video-scoring-ui:latest .
```

- `-t video-scoring-ui:latest`: Gán tên `video-scoring-ui` với tag `latest` cho image.
- `.`: Chỉ định context build là thư mục hiện tại.

## 7.4. Chạy Docker Container

Sau khi build xong image, bạn có thể chạy container bằng lệnh:

```bash
docker run -d -p 80:80 --name video-scoring-ui-container video-scoring-ui:latest
```

- `-d`: Chạy container ở chế độ detached (nền).
- `-p 80:80`: Map cổng 80 của host đến cổng 80 của container.
- `--name video-scoring-ui-container`: Đặt tên cho container.
- `video-scoring-ui:latest`: Tên và tag của image sẽ chạy.

Bây giờ, ứng dụng React của bạn sẽ được phục vụ bởi Nginx và có thể truy cập tại `http://localhost`.

## 7.5. Sử dụng Docker Compose (Tùy chọn)

Nếu bạn muốn quản lý container dễ dàng hơn, bạn có thể sử dụng Docker Compose. Tạo một file `docker-compose.yml` trong thư mục gốc của project với nội dung sau:

```yaml
version: '3.8'

services:
  ui_react:
    build:
      context: .
      dockerfile: Dockerfile
    ports:
      - "80:80"
    container_name: video-scoring-ui-container
    restart: unless-stopped
```

### Các bước sử dụng Docker Compose:

1. **Build và chạy container:**

   ```bash
   docker-compose up -d
   ```

   - `-d`: Chạy dịch vụ ở chế độ detached.

2. **Dừng và xóa container:**

   ```bash
   docker-compose down
   ```

3. **Xây dựng lại image (khi có thay đổi):**

   ```bash
   docker-compose up -d --build
   ```

## 7.6. Kiểm Tra

Sau khi container đang chạy, mở trình duyệt và truy cập vào `http://localhost` để xem ứng dụng React của bạn đang hoạt động.

## 7.7. Lưu ý

- **Port**: Đảm bảo rằng cổng 80 không bị chiếm dụng bởi ứng dụng hoặc dịch vụ khác trên máy của bạn. Nếu cần, bạn có thể thay đổi cổng host trong `docker run` hoặc `docker-compose.yml`, ví dụ `-p 3000:80` để truy cập tại `http://localhost:3000`.
  
- **Build nhanh hơn trong quá trình phát triển**: Trong quá trình phát triển, bạn có thể muốn mount mã nguồn vào container để tự động cập nhật khi thay đổi mã. Dưới đây là một ví dụ về `docker-compose.yml` để hỗ trợ phát triển:

  ```yaml
  version: '3.8'

  services:
    ui_react:
      image: node:18-alpine
      working_dir: /app
      volumes:
        - .:/app
        - /app/node_modules
      ports:
        - "3000:3000"
      command: npm start
      environment:
        - NODE_ENV=development
  ```

  Với cấu hình này, bạn có thể truy cập ứng dụng tại `http://localhost:3000` và mọi thay đổi mã nguồn sẽ tự động được phản ánh mà không cần rebuild image.

## 7.8. Tài Nguyên Tham Khảo

- [Docker Documentation](https://docs.docker.com/)
- [Docker Compose Documentation](https://docs.docker.com/compose/)
- [Create React App Deployment](https://create-react-app.dev/docs/deployment/)

Hy vọng rằng hướng dẫn trên sẽ giúp bạn dễ dàng triển khai và chạy ứng dụng React của mình bằng Docker. Nếu gặp bất kỳ vấn đề nào, hãy kiểm tra lại các bước cấu hình hoặc tham khảo tài liệu chính thức của Docker.

THÀNH CÔNG 